In [ ]:
import ipywidgets.widgets as widgets
from time import sleep, time
import sys
sys.path.append('/home/ballsbot/projects/ballsbot/python/lib')
from ballsbot.drawing import BotPoseDrawing
from ballsbot.camera import get_images_and_cameras
from ballsbot.servos import get_controls
from ballsbot.utils import keep_rps
from ballsbot.lidar import calibration_to_xywh
from ballsbot.lidar_with_memory import LidarWithMemory
from ballsbot.utils import run_as_thread
from ballsbot.tracking import Tracker
from ballsbot.controller import link_controller
from ballsbot_localization import ballsbot_localization as grid
from ballsbot import drawing
from ballsbot.ros_messages import get_ros_messages

In [ ]:
images = []
for image, camera in get_images_and_cameras():
    images.append(image)
hbox = widgets.HBox(images)
display(hbox)

track_image = widgets.Image(format='jpeg', width=600, height=600)
display(track_image)

controller = widgets.Controller(index=0)  # http://html5gamepad.com
display(controller)

In [ ]:
link_controller(controller)

painter = BotPoseDrawing(track_image)

lidar = LidarWithMemory()
tracker = Tracker(lidar)
tracker.start()

get_ros_messages().start(sync=False)

In [ ]:
def update_track():
    ts = None
    while True:
        ts = keep_rps(ts, fps=2)
        params = tracker.get_picture_params(with_free_tiles=True)
        if params:
            painter.update_image(*params, only_nearby_meters=6.)

run_as_thread(update_track)

In [ ]:
# from ballsbot.utils import join_all_theads
# join_all_theads()

In [ ]:
# from ballsbot_localization import ballsbot_localization as grid
# grid.get_poses()

# import json
# print(json.dumps({
#     "frames_processed": tracker.lidar_frames_processed,
#     "lidar_deque": len(tracker.lidar_deque),
#     "lidar_deque_empty_times": tracker.lidar_deque_empty_times,
#     "pose_deque": len(tracker.pose_deque),
#     "pose_deque_empty_times": tracker.pose_deque_empty_times,
#     "distances_ignored_count": tracker.lidar.lidar.distances_ignored_count,
#     "distances_ignored_old_count": tracker.lidar.lidar.distances_ignored_old_count,
#     "distances_ignored_new_count": tracker.lidar.lidar.distances_ignored_new_count,
#     "distances_merged_count": tracker.lidar.lidar.distances_merged_count,
# }, indent=4, sort_keys=True))

# from time import time
# lidar_ts = tracker.lidar_deque[0]['ts']
# pose_ts = tracker.pose_deque[0]['ts']
# real_ts = time()
# (real_ts - lidar_ts, real_ts - pose_ts)

In [ ]:
# import json
# from ballsbot_localization import ballsbot_localization as grid
# from ballsbot.config import TURN_DIAMETER, FROM_LIDAR_TO_CENTER, CAR_WIDTH, CAR_LENGTH, ENGINE_NEED_MANUAL_BREAKING, FROM_LIDAR_TO_PIVOT_CENTER

# grid.debug_get_free_distances(
#     lidar.get_points_ts(),
#     {
#         'to_car_center': FROM_LIDAR_TO_CENTER,
#         'turn_radius': TURN_DIAMETER / 2.,
#         'to_pivot_center': FROM_LIDAR_TO_PIVOT_CENTER,
#         'car_width': CAR_WIDTH,
#         'car_length': CAR_LENGTH,
#         'engine_need_manual_breaking': ENGINE_NEED_MANUAL_BREAKING,
#     }
# )

# state = tracker.lidar.tick_get_points()
# state.update({
#     'pose': tracker.get_current_pose(),
#     'car_info': {
#         'to_car_center': FROM_LIDAR_TO_CENTER,
#         'turn_radius': TURN_DIAMETER / 2.,
#         'to_pivot_center': FROM_LIDAR_TO_PIVOT_CENTER,
#         'car_width': CAR_WIDTH,
#         'car_length': CAR_LENGTH,
#         'engine_need_manual_breaking': ENGINE_NEED_MANUAL_BREAKING,
#     },
# })
# with open('/home/ballsbot/test.json', 'w') as hf:
#     json.dump(state, hf, indent=4, sort_keys=True)